In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/school/DSA3101/

/content/drive/My Drive/school/DSA3101


In [ ]:
import pandas as pd

# Preparing data for merge

In [ ]:
scraped_df = pd.read_csv('final_data_modified.csv')
scraped_df = scraped_df.loc[:, ~scraped_df.columns.isin(['developer_response'])]
scraped_df

,rowid,date,client,bank,rating,review
0,0,2024-03-06 08:36:37,Apple,GXS,4.0,Great banking app with attractive interest rat...
1,1,2022-11-09 03:13:16,Apple,GXS,5.0,"A bank like no other, no bank have such amazin..."
2,2,2023-09-09 08:33:43,Apple,GXS,1.0,Notice that the drop in interest rate of 0.8% ...
3,3,2023-06-27 04:01:39,Apple,GXS,2.0,Sending money into my GXS account is a breeze ...
4,4,2022-10-02 04:17:49,Apple,GXS,5.0,I have to say that the UI/UX is one of the bes...
...,...,...,...,...,...,...
1089,1089,2023-12-26 10:31:11,Android,Trust,1.0,"After the latest update, I can't even open the..."
1090,1090,2023-12-26 03:54:34,Android,Trust,5.0,How to pay my bill for trust card
1091,1091,2023-12-23 13:20:31,Android,Trust,3.0,the app indicates exchange rate in trust is be...
1092,1092,2023-12-22 18:22:03,Android,Trust,1.0,The app is in conflict with my phone voice rec...


In [ ]:
processed_df = pd.read_csv("sentiment_service_issue.csv")
processed_df = processed_df.loc[:, ~processed_df.columns.isin(['bank'])]
processed_df

,rowid,sentiment,problem_category,issue
0,0,Positive,app,NaN
1,1,Positive,banking,NaN
2,2,Neutral,banking,Interest rate concerns
3,3,Positive,app,NaN
4,4,Positive,app,NaN
...,...,...,...,...
1068,1089,Negative,app,technical glitches
1069,1090,Neutral,app,Difficulty applying credit card
1070,1091,Neutral,app,other
1071,1092,Negative,app,other


# 1. Join sentiment, intent, service, issue with scraped data

In [ ]:
# merge on common key rowid
merged_df = pd.merge(scraped_df, processed_df, on = 'rowid')

merged_df.rename(columns = {'problem_category': 'service'}, inplace=True)
merged_df['sentiment'] = merged_df['sentiment'].replace([' Negative', ' Neutral', ' Neutural', ' Positive'], ['Negative', 'Neutral', 'Neutral', 'Positive'])

merged_df['issue'] = merged_df['issue'].replace(
    ['lack_of_expected_features_or_functionality',
    'technical difficulties during registration and setup',
    'technical issues',
    'user interface and design issues',
    'Account Access Issues',
    'Account issues',
    'Interest rate concerns',
    'login problems'],

     ['lack of expected features or functionality',
      'registration and setup',
      'technical glitches',
      'UI/UX',
      'account issues',
      'account issues',
      'interest rate concerns',
      'login issues']
    )

merged_df

,rowid,date,client,bank,rating,review,sentiment,service,issue
0,0,2024-03-06 08:36:37,Apple,GXS,4.0,Great banking app with attractive interest rat...,Positive,app,NaN
1,1,2022-11-09 03:13:16,Apple,GXS,5.0,"A bank like no other, no bank have such amazin...",Positive,banking,NaN
2,2,2023-09-09 08:33:43,Apple,GXS,1.0,Notice that the drop in interest rate of 0.8% ...,Neutral,banking,interest rate concerns
3,3,2023-06-27 04:01:39,Apple,GXS,2.0,Sending money into my GXS account is a breeze ...,Positive,app,NaN
4,4,2022-10-02 04:17:49,Apple,GXS,5.0,I have to say that the UI/UX is one of the bes...,Positive,app,NaN
...,...,...,...,...,...,...,...,...,...
1068,1089,2023-12-26 10:31:11,Android,Trust,1.0,"After the latest update, I can't even open the...",Negative,app,technical glitches
1069,1090,2023-12-26 03:54:34,Android,Trust,5.0,How to pay my bill for trust card,Neutral,app,Difficulty applying credit card
1070,1091,2023-12-23 13:20:31,Android,Trust,3.0,the app indicates exchange rate in trust is be...,Neutral,app,other
1071,1092,2023-12-22 18:22:03,Android,Trust,1.0,The app is in conflict with my phone voice rec...,Negative,app,other


# 2. Find top 2 issues within GXS

filter GXS data only

In [ ]:
gxs = merged_df[merged_df['bank'] == 'GXS']
gxs

,rowid,date,client,bank,rating,review,sentiment,service,issue
0,0,2024-03-06 08:36:37,Apple,GXS,4.0,Great banking app with attractive interest rat...,Positive,app,NaN
1,1,2022-11-09 03:13:16,Apple,GXS,5.0,"A bank like no other, no bank have such amazin...",Positive,banking,NaN
2,2,2023-09-09 08:33:43,Apple,GXS,1.0,Notice that the drop in interest rate of 0.8% ...,Neutral,banking,interest rate concerns
3,3,2023-06-27 04:01:39,Apple,GXS,2.0,Sending money into my GXS account is a breeze ...,Positive,app,NaN
4,4,2022-10-02 04:17:49,Apple,GXS,5.0,I have to say that the UI/UX is one of the bes...,Positive,app,NaN
...,...,...,...,...,...,...,...,...,...
869,890,2023-07-10 16:42:56,Android,GXS,1.0,Have been downloading this app for past few da...,Neutral,app,technical glitches
870,891,2023-07-09 19:51:49,Android,GXS,1.0,Misleading advertisement for flexiloan promoti...,Negative,app,other
871,892,2023-07-06 09:19:04,Android,GXS,1.0,Can't even load properly. Prompted to set 6 di...,Negative,app,registration and setup
872,893,2023-07-04 13:47:06,Android,GXS,5.0,Easy to use. Awesome,Positive,app,NaN


In [ ]:
#gxs_issue_by_count = gxs.groupby('issue').apply(issue_count_agg_function).dropna()
#gxs_issue_by_count

,,issue,count
issue,,,
Difficulty applying,0,Difficulty applying,1
"Difficulty applying credit card, savings account",0,"Difficulty applying credit card, savings account",1
Difficulty applying savings account,0,Difficulty applying savings account,1
Long Wait Times,0,Long Wait Times,5
UI/UX,0,UI/UX,6
account issues,0,account issues,8
application rejection,0,application rejection,6
interest rate concerns,0,interest rate concerns,14
lack of expected features or functionality,0,lack of expected features or functionality,7


filter negative sentiment

In [ ]:
gxs_neg = gxs[gxs['sentiment'] == 'Negative']
# Reset the index if needed
gxs_neg.reset_index(drop=True, inplace=True)
gxs_neg

,rowid,date,client,bank,rating,review,sentiment,service,issue
0,7,2023-09-13 15:30:05,Apple,GXS,1.0,Hit with fast reversal when transferring money...,Negative,banking,technical glitches
1,9,2022-09-08 07:14:48,Apple,GXS,1.0,What a joke! Nothing happens after you input y...,Negative,app,other
2,11,2023-08-22 10:22:46,Apple,GXS,1.0,My biggest frustration is that I can’t save pa...,Negative,app,Difficulty applying savings account
3,14,2022-10-31 05:18:02,Apple,GXS,5.0,Despite a lengthy onboarding but my account go...,Negative,banking,none
4,18,2023-09-12 05:10:29,Apple,GXS,1.0,Already 1 year but basic functions like adding...,Negative,app,lack of expected features or functionality
...,...,...,...,...,...,...,...,...,...
149,885,2023-07-17 13:27:25,Android,GXS,1.0,Worst apps everything also no resolve well,Negative,app,other
150,886,2023-07-15 02:18:08,Android,GXS,1.0,Waste of time app!! Keep on crashing when want...,Negative,app,technical glitches
151,887,2023-07-14 06:44:30,Android,GXS,3.0,How to cancel or delete all my personal detail...,Negative,banking,other
152,891,2023-07-09 19:51:49,Android,GXS,1.0,Misleading advertisement for flexiloan promoti...,Negative,app,other


aggregate count of service

In [ ]:
# Define a custom aggregation function that returns a DataFrame
def count_agg_function(group):
    # Get the group column value for this group
    group_value = group['service'].iloc[0]
    print(group_value)
    # Perform aggregation operations and return a DataFrame
    aggregated_df = pd.DataFrame({
        'service': [group_value],
        'count': [group['rowid'].count()]
    })
    return aggregated_df

In [ ]:
# Aggregate the DataFrame by a specific column
gxs_neg_service_by_count = gxs_neg.groupby('service').apply(count_agg_function)
# Reset the index if needed
gxs_neg_service_by_count.reset_index(drop=True, inplace=True)

gxs_neg_service_by_count

app
banking
signup


,service,count
0,app,115
1,banking,38
2,signup,1


get service 1 and 2

In [ ]:
# Find the row with the maximum count in the 'count_column'
max_count_row = gxs_neg_service_by_count.loc[gxs_neg_service_by_count['count'].idxmax()]

# Get the variable with the most count
service_1 = max_count_row['service']

# Sort the aggregated DataFrame by 'count_column' in descending order
sorted_aggregated_df = gxs_neg_service_by_count.sort_values(by='count', ascending=False)

# Get the variable with the second highest count
service_2 = sorted_aggregated_df.iloc[1]['service']

print(service_1, service_2)

app banking


filter service 1

In [ ]:
service_1_df = gxs_neg[gxs_neg['service'] == service_1]

# Reset the index if needed
service_1_df.reset_index(drop=True, inplace=True)
service_1_df.head()

,rowid,date,client,bank,rating,review,sentiment,service,issue
0,9,2022-09-08 07:14:48,Apple,GXS,1.0,What a joke! Nothing happens after you input y...,Negative,app,other
1,11,2023-08-22 10:22:46,Apple,GXS,1.0,My biggest frustration is that I can’t save pa...,Negative,app,Difficulty applying savings account
2,18,2023-09-12 05:10:29,Apple,GXS,1.0,Already 1 year but basic functions like adding...,Negative,app,lack of expected features or functionality
3,19,2024-02-24 06:12:02,Apple,GXS,1.0,Extremely disappointed with the GXS app. Despi...,Negative,app,other
4,29,2022-09-06 17:57:25,Apple,GXS,2.0,Downloaded the app but doesnt seem to be able ...,Negative,app,registration and setup


aggregate count of service 1

In [ ]:
# Define a custom aggregation function that returns a DataFrame
def issue_count_agg_function(group):
    # Initialize an empty list to store the results
    results = []

    # Iterate over all the values in the 'issue' column for this group
    for group_value in group['issue'].unique():
        # Check if group_value is not None
        if pd.notna(group_value):
            # Perform aggregation operations and append the result to the list
            aggregated_result = {
                'issue': group_value,
                'count': group.loc[group['issue'] == group_value, 'rowid'].count()
                # Add more aggregation operations as needed
            }
            results.append(aggregated_result)

    # Convert the list of results to a DataFrame
    aggregated_df = pd.DataFrame(results)

    return aggregated_df


In [ ]:
# Aggregate the DataFrame by a specific column
gxs_neg_issue_by_count_1 = service_1_df.groupby('issue').apply(issue_count_agg_function).dropna()
gxs_neg_issue_by_count_1_no_other = gxs_neg_issue_by_count_1[gxs_neg_issue_by_count_1['issue'] != 'other']
gxs_neg_issue_by_count_1_no_other = gxs_neg_issue_by_count_1_no_other.sort_values(by = 'count', ascending = False)

# Reset the index if needed
gxs_neg_issue_by_count_1_no_other.reset_index(drop=True, inplace=True)

gxs_neg_issue_by_count_1_no_other

,issue,count
0,technical glitches,24
1,registration and setup,20
2,login issues,16
3,lack of expected features or functionality,5
4,UI/UX,2
5,account issues,2
6,"Difficulty applying credit card, savings account",1
7,Difficulty applying savings account,1
8,Long Wait Times,1


get top 2 issues (issue 1 and issue 2)

In [ ]:
# Find the 2 issues with the maximum count in the 'count_column'
issue_1 = gxs_neg_issue_by_count_1_no_other['issue'].values[0]
issue_2 = gxs_neg_issue_by_count_1_no_other['issue'].values[1]

print(issue_1, ',', issue_2)

technical glitches , registration and setup


filter service 2

In [ ]:
#service_2_df = gxs_neg[gxs_neg['service'] == service_2]
# Reset the index if needed
#service_2_df.reset_index(drop=True, inplace=True)
#service_2_df.head()

aggregate count of service 2

In [ ]:
# Aggregate the DataFrame by a specific column
#gxs_neg_issue_by_count_2 = service_2_df.groupby('issue').apply(issue_count_agg_function).dropna()
#gxs_neg_issue_by_count_2_no_other = gxs_neg_issue_by_count_2[gxs_neg_issue_by_count_2['issue'] != 'other']
# Reset the index if needed
#gxs_neg_issue_by_count_2_no_other.reset_index(drop=True, inplace=True)

#gxs_neg_issue_by_count_2_no_other

,issue,count
0,Account issues,1
1,Interest rate concerns,5
2,Long Wait Times,2
3,account issues,5
4,application rejection,3
5,interest rate concerns,3
6,login problems,1
7,none,1
8,technical glitches,8
9,technical issues,1


get issue 2

In [ ]:
# Find the row with the maximum count in the 'count_column'
#max_count_row_issue_2 = gxs_neg_issue_by_count_2_no_other.loc[gxs_neg_issue_by_count_2_no_other['count'].idxmax()]

# Get the variable with the most count
#issue_2 = max_count_row_issue_2['issue']
#issue_2

'technical glitches'

# 2a. Issue #1

filter issue 1

In [ ]:
issue_1_df = gxs_neg[(gxs_neg['service'] == service_1) & (gxs_neg['issue'] == issue_1)]
# Reset the index if needed
issue_1_df.reset_index(drop=True, inplace=True)
issue_1_df

,rowid,date,client,bank,rating,review,sentiment,service,issue
0,74,2023-07-21 23:56:46,Apple,GXS,1.0,Extremely disappointed after downloading the a...,Negative,app,technical glitches
1,75,2023-11-15 11:32:12,Apple,GXS,1.0,"I cannot open the app at all, even after updat...",Negative,app,technical glitches
2,709,2024-02-24 03:52:03,Android,GXS,1.0,Can not open,Negative,app,technical glitches
3,731,2023-12-18 11:13:13,Android,GXS,1.0,could not open my gxs app,Negative,app,technical glitches
4,739,2023-12-06 07:12:41,Android,GXS,1.0,Couldn't use the app suddenly,Negative,app,technical glitches
5,774,2023-10-07 14:03:38,Android,GXS,1.0,"Pathetic app and support. Paynow qr, which is ...",Negative,app,technical glitches
6,790,2023-09-11 14:16:16,Android,GXS,1.0,Can't even open the GXS app... the only way to...,Negative,app,technical glitches
7,791,2023-09-10 10:08:38,Android,GXS,1.0,Attempt to try but it keep post something wron...,Negative,app,technical glitches
8,792,2023-09-10 04:24:08,Android,GXS,1.0,Unable to open the app and use my funds. Is th...,Negative,app,technical glitches
9,795,2023-09-07 22:35:27,Android,GXS,1.0,"Stuck on Oops, something went wrong page when ...",Negative,app,technical glitches


give filtered table with rowid, text data of issue 1 for recommendation

In [ ]:
issue_1_df['recommendation_no'] = 'R1'
issue_1_recommend = issue_1_df[['rowid', 'service', 'issue','recommendation_no','review']]
issue_1_recommend

<ipython-input-233-ecff9ca947c7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  issue_1_df['recommendation_no'] = 'R1'


,rowid,service,issue,recommendation_no,review
0,74,app,technical glitches,R1,Extremely disappointed after downloading the a...
1,75,app,technical glitches,R1,"I cannot open the app at all, even after updat..."
2,709,app,technical glitches,R1,Can not open
3,731,app,technical glitches,R1,could not open my gxs app
4,739,app,technical glitches,R1,Couldn't use the app suddenly
5,774,app,technical glitches,R1,"Pathetic app and support. Paynow qr, which is ..."
6,790,app,technical glitches,R1,Can't even open the GXS app... the only way to...
7,791,app,technical glitches,R1,Attempt to try but it keep post something wron...
8,792,app,technical glitches,R1,Unable to open the app and use my funds. Is th...
9,795,app,technical glitches,R1,"Stuck on Oops, something went wrong page when ..."


# 2b. Issue #2

filter issue 2

In [ ]:
issue_2_df = gxs_neg[(gxs_neg['service'] == service_1) & (gxs_neg['issue'] == issue_2)]
# Reset the index if needed
issue_2_df.reset_index(drop=True, inplace=True)
issue_2_df

,rowid,date,client,bank,rating,review,sentiment,service,issue
0,29,2022-09-06 17:57:25,Apple,GXS,2.0,Downloaded the app but doesnt seem to be able ...,Negative,app,registration and setup
1,96,2023-11-23 09:53:53,Apple,GXS,1.0,When I sign up can’t sign up always say oops s...,Negative,app,registration and setup
2,118,2023-11-02 06:38:15,Apple,GXS,1.0,"useless app. unable to join or open account,\...",Negative,app,registration and setup
3,718,2024-01-12 15:34:50,Android,GXS,5.0,I don't set. A password in my phone n not allo...,Negative,app,registration and setup
4,722,2023-12-30 10:08:20,Android,GXS,1.0,I'm stuck on the singpass page without being a...,Negative,app,registration and setup
5,724,2023-12-28 08:09:13,Android,GXS,1.0,Cannot sig up,Negative,app,registration and setup
6,749,2023-11-16 05:01:32,Android,GXS,1.0,Cannot set up my account in redmi 8T. Reach th...,Negative,app,registration and setup
7,772,2023-10-08 14:02:46,Android,GXS,1.0,Seems to be full. Unable to sign up. Uninstall...,Negative,app,registration and setup
8,784,2023-09-18 09:30:25,Android,GXS,1.0,"Cannot register, keep nothing pop up On sign up",Negative,app,registration and setup
9,811,2023-08-25 00:27:39,Android,GXS,1.0,Have not even tried already failed on me. Just...,Negative,app,registration and setup


give filtered table with rowid, text data of issue 2 for recommendation

In [ ]:
issue_2_df['recommendation_no'] = 'R2'
issue_2_recommend = issue_2_df[['rowid', 'service', 'issue', 'recommendation_no', 'review']]
issue_2_recommend

<ipython-input-235-2104786eefb2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  issue_2_df['recommendation_no'] = 'R2'


,rowid,service,issue,recommendation_no,review
0,29,app,registration and setup,R2,Downloaded the app but doesnt seem to be able ...
1,96,app,registration and setup,R2,When I sign up can’t sign up always say oops s...
2,118,app,registration and setup,R2,"useless app. unable to join or open account,\..."
3,718,app,registration and setup,R2,I don't set. A password in my phone n not allo...
4,722,app,registration and setup,R2,I'm stuck on the singpass page without being a...
5,724,app,registration and setup,R2,Cannot sig up
6,749,app,registration and setup,R2,Cannot set up my account in redmi 8T. Reach th...
7,772,app,registration and setup,R2,Seems to be full. Unable to sign up. Uninstall...
8,784,app,registration and setup,R2,"Cannot register, keep nothing pop up On sign up"
9,811,app,registration and setup,R2,Have not even tried already failed on me. Just...


# 3. Find top issue to keep up with/continue leading Trust Bank

filter Trust bank data only

In [ ]:
trust = merged_df[merged_df['bank'] == 'Trust'].dropna()
trust['service'] = trust['service'].replace('bank', 'banking')
trust.reset_index(drop=True, inplace=True)
trust

,rowid,date,client,bank,rating,review,sentiment,service,issue
0,132,2022-11-08 02:34:34,Apple,Trust,5.0,"I had my card locked the night before, and con...",Negative,banking,Long Wait Times
1,134,2022-09-07 05:38:04,Apple,Trust,1.0,"On 04Sept, i accidentally clicked “yes” to be ...",Neutral,banking,Long Wait Times
2,135,2023-08-22 15:41:09,Apple,Trust,1.0,The interface is not intuitive at all. You lit...,Negative,app,UI/UX
3,136,2023-02-15 14:58:27,Apple,Trust,5.0,"Hi, I’ve been using trust bank for a while now...",Neutral,banking,other
4,137,2023-10-23 14:01:00,Apple,Trust,1.0,If you’re a first-time salaried employee and w...,Neutral,banking,account issues
...,...,...,...,...,...,...,...,...,...
493,1089,2023-12-26 10:31:11,Android,Trust,1.0,"After the latest update, I can't even open the...",Negative,app,technical glitches
494,1090,2023-12-26 03:54:34,Android,Trust,5.0,How to pay my bill for trust card,Neutral,app,Difficulty applying credit card
495,1091,2023-12-23 13:20:31,Android,Trust,3.0,the app indicates exchange rate in trust is be...,Neutral,app,other
496,1092,2023-12-22 18:22:03,Android,Trust,1.0,The app is in conflict with my phone voice rec...,Negative,app,other



filter negative sentiment

In [ ]:
trust_neg = trust[trust['sentiment'] == 'Negative']
trust_neg.reset_index(drop=True, inplace=True)
trust_neg.head()

,rowid,date,client,bank,rating,review,sentiment,service,issue
0,132,2022-11-08 02:34:34,Apple,Trust,5.0,"I had my card locked the night before, and con...",Negative,banking,Long Wait Times
1,135,2023-08-22 15:41:09,Apple,Trust,1.0,The interface is not intuitive at all. You lit...,Negative,app,UI/UX
2,143,2022-09-11 04:22:51,Apple,Trust,1.0,I will be critical base on the pointers :\n\n1...,Negative,app,Long Wait Times
3,146,2024-01-03 03:52:13,Apple,Trust,3.0,My account was locked as there is multiple tri...,Negative,banking,account issues
4,148,2023-02-21 06:57:21,Apple,Trust,2.0,"If it is a digital bank for everyone, why one ...",Negative,app,other


aggregate count of service

In [ ]:
# Aggregate the DataFrame by a specific column
trust_neg_service_by_count = trust_neg.groupby('service').apply(count_agg_function)

# Reset the index if needed
trust_neg_service_by_count.reset_index(drop=True, inplace=True)

trust_neg_service_by_count

app
banking


,service,count
0,app,214
1,banking,108


In [ ]:
# Find the row with the minimum count in the 'count_column'
min_count_row_service_3 = trust_neg_service_by_count.loc[trust_neg_service_by_count['count'].idxmin()]

# Get the variable with the least count
service_3 = min_count_row_service_3['service']
service_3

'banking'

filter service 3

In [ ]:
service_3_df = trust_neg[trust_neg['service'] == service_3]
# Reset the index if needed
service_3_df.reset_index(drop=True, inplace=True)
service_3_df

,rowid,date,client,bank,rating,review,sentiment,service,issue
0,132,2022-11-08 02:34:34,Apple,Trust,5.0,"I had my card locked the night before, and con...",Negative,banking,Long Wait Times
1,146,2024-01-03 03:52:13,Apple,Trust,3.0,My account was locked as there is multiple tri...,Negative,banking,account issues
2,160,2023-04-27 00:30:03,Apple,Trust,1.0,"App is not the issue, the bank and service is ...",Negative,banking,other
3,163,2022-11-19 14:16:36,Apple,Trust,1.0,Been facing double charging for the past 2 day...,Negative,banking,transaction issues
4,165,2022-09-20 08:28:31,Apple,Trust,2.0,Application pending for about 2 weeks already....,Negative,banking,Long Wait Times
...,...,...,...,...,...,...,...,...,...
103,1054,2024-01-18 08:32:25,Android,Trust,1.0,very disappointing.Trust bank does not have we...,Negative,banking,other
104,1057,2024-01-16 03:11:11,Android,Trust,2.0,"While the TrustBank is promising, I'm afraid t...",Negative,banking,technical glitches
105,1058,2024-01-15 16:21:57,Android,Trust,1.0,Scam app beware folks,Negative,banking,other
106,1071,2024-01-09 22:26:56,Android,Trust,2.0,Posted transactions take too long to be confir...,Negative,banking,other


aggregate count of service 3

In [ ]:
# Aggregate the DataFrame by a specific column
trust_neg_issue_by_count = service_3_df.groupby('issue').apply(issue_count_agg_function).dropna()
trust_neg_issue_by_count_no_other = trust_neg_issue_by_count[(trust_neg_issue_by_count['issue'] != 'other')]
trust_neg_issue_by_count_no_other = trust_neg_issue_by_count_no_other.sort_values(by = 'count')
# Reset the index if needed
trust_neg_issue_by_count_no_other.reset_index(drop=True, inplace=True)

trust_neg_issue_by_count_no_other

,issue,count
0,UI/UX,1
1,account creation,1
2,fraudulent,1
3,unauthorized transactions,1
4,application rejection,4
5,interest rate concerns,4
6,transaction issues,4
7,account issues,9
8,Long Wait Times,19
9,technical glitches,21


get issue 3

In [ ]:
# Find the row with the minimum count in the 'count_column'
min_count_row_issue_3 = trust_neg_issue_by_count_no_other.loc[trust_neg_issue_by_count_no_other['count'].idxmin()]

# Get the variable with the most count
issue_3 = min_count_row_issue_3['issue']
issue_3

'UI/UX'

get reviews with issue 3 in gxs

In [ ]:
issue_3_df = gxs[gxs['issue'] == issue_3]
# Reset the index if needed
issue_3_df.reset_index(drop=True, inplace=True)
issue_3_df

,rowid,date,client,bank,rating,review,sentiment,service,issue
0,60,2022-12-23 04:53:44,Apple,GXS,3.0,"Personally don’t like the theme, anyway allow ...",Negative,app,UI/UX
1,756,2023-11-10 20:48:55,Android,GXS,5.0,Ease of use.,Neutral,app,UI/UX
2,759,2023-11-10 07:19:33,Android,GXS,5.0,Easy of use,Neutral,app,UI/UX
3,760,2023-11-10 07:07:05,Android,GXS,5.0,Ease of use,Neutral,app,UI/UX
4,849,2023-07-26 09:43:35,Android,GXS,1.0,App is not clear and Singpass details are not ...,Negative,app,UI/UX


give filtered table with rowid, text data of issue 3 for recommendation

In [ ]:
issue_3_df['recommendation_no'] = 'R3'
issue_3_recommend = issue_3_df[['rowid', 'service', 'issue','recommendation_no','review']]
issue_3_recommend

<ipython-input-244-62c6ddd7dc04>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  issue_3_df['recommendation_no'] = 'R3'


,rowid,service,issue,recommendation_no,review
0,60,app,UI/UX,R3,"Personally don’t like the theme, anyway allow ..."
1,756,app,UI/UX,R3,Ease of use.
2,759,app,UI/UX,R3,Easy of use
3,760,app,UI/UX,R3,Ease of use
4,849,app,UI/UX,R3,App is not clear and Singpass details are not ...


# 4. Merge the 3 dfs for recommendation

In [ ]:
combine_issue_recommend = pd.concat([issue_1_recommend, issue_2_recommend, issue_3_recommend])
combine_issue_recommend.reset_index(drop=True, inplace=True)
combine_issue_recommend

,rowid,service,issue,recommendation_no,review
0,74,app,technical glitches,R1,Extremely disappointed after downloading the a...
1,75,app,technical glitches,R1,"I cannot open the app at all, even after updat..."
2,709,app,technical glitches,R1,Can not open
3,731,app,technical glitches,R1,could not open my gxs app
4,739,app,technical glitches,R1,Couldn't use the app suddenly
5,774,app,technical glitches,R1,"Pathetic app and support. Paynow qr, which is ..."
6,790,app,technical glitches,R1,Can't even open the GXS app... the only way to...
7,791,app,technical glitches,R1,Attempt to try but it keep post something wron...
8,792,app,technical glitches,R1,Unable to open the app and use my funds. Is th...
9,795,app,technical glitches,R1,"Stuck on Oops, something went wrong page when ..."


export to csv

In [ ]:
combine_issue_recommend.to_csv("eda_for_reco.csv", index = False)

from google.colab import files
files.download('eda_for_reco.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>